## 1. Import Library

    Langkah pertama adalah mengimpor semua pustaka Python yang kita butuhkan untuk analisis data, dan pemrosesan geospasial.

In [18]:
import pandas as pd
import geopandas as gpd
import os
import glob
import numpy as np

## 2. Membaca Data Geospasial (Batas Wilayah)

    Muat file GeoJSON yang berisi data poligon batas wilayah administrasi kabupaten di Kalimantan Tengah. Ini akan digunakan untuk menandai lokasi setiap hotspot.

In [19]:
file_geojson = 'E:/SPARK/SPARK/data/geo/kabupaten.geojson'

try:
    gdf_kalteng = gpd.read_file(file_geojson)
    # Standarisasi nama kolom kabupaten menjadi huruf kapital untuk konsistensi
    gdf_kalteng['NAME_2'] = gdf_kalteng['NAME_2'].str.upper()
    print("File GeoJSON berhasil dibaca.")
    print("Contoh nama kabupaten:", gdf_kalteng['NAME_2'].unique()[:5])
except Exception as e:
    print(f"ERROR: Gagal membaca file GeoJSON. Detail: {e}")
    gdf_kalteng = gpd.GeoDataFrame()

File GeoJSON berhasil dibaca.
Contoh nama kabupaten: ['BARITO SELATAN' 'BARITO TIMUR' 'BARITO UTARA' 'GUNUNGMAS' 'KAPUAS']


## 3. Menggabungkan Semua Data Cuaca

    Ini secara otomatis mencari dan menggabungkan semua file data cuaca dari tahun 2018-2022 dari folder masing-masing kabupaten, sekaligus membersihkannya.

In [20]:
base_path = 'E:/SPARK/SPARK/data/raw/Data Iklim Kalimantan Tengah'
list_folder_kabupaten = [
    'Barito Selatan', 'Barito Utara', 'Kotawaringin Barat', 
    'Kotawaringin Timur', 'Palangkaraya'
]

all_excel_files = []
for folder in list_folder_kabupaten:
    search_pattern = os.path.join(base_path, folder, '**', '*.xlsx')
    all_excel_files.extend(glob.glob(search_pattern, recursive=True))

list_df_cuaca = []
for file in all_excel_files:
    try:
        df = pd.read_excel(file, skiprows=lambda x: x in [0,1,2,3,4,5,6,8])
        df.columns = df.columns.str.strip().str.lower()
        
        df['tanggal'] = pd.to_datetime(df['tanggal'], errors='coerce')
        df.dropna(subset=['tanggal'], inplace=True)
        
        df.replace(['8888', 9988, 9999, '-', ' '], np.nan, inplace=True)

        kabupaten_name = os.path.normpath(os.path.relpath(file, base_path)).split(os.sep)[0]
        df['KABUPATEN'] = kabupaten_name
        list_df_cuaca.append(df)
    except Exception as e:
        print(f"  - Gagal memproses file {file}: {e}")

if list_df_cuaca:
    df_cuaca_all = pd.concat(list_df_cuaca, ignore_index=True)
    df_cuaca_all.rename(columns={'tanggal': 'DATA_TIMESTAMP'}, inplace=True)
    df_cuaca_all['KABUPATEN'] = df_cuaca_all['KABUPATEN'].str.upper().str.replace('-', ' ').str.replace('_', ' ')

    for col in ['tavg', 'tx', 'tn', 'rh_avg', 'rr', 'ss']:
        if col in df_cuaca_all.columns:
            df_cuaca_all[col] = pd.to_numeric(df_cuaca_all[col], errors='coerce')
            
    print("\nSemua data cuaca berhasil digabungkan dan dibersihkan.")
    print(df_cuaca_all.info())
else:
    print("\nTidak ada data cuaca yang berhasil diproses.")
    df_cuaca_all = pd.DataFrame()

C:\Users\IRHAM\AppData\Local\Temp\ipykernel_11628\578528262.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['8888', 9988, 9999, '-', ' '], np.nan, inplace=True)
C:\Users\IRHAM\AppData\Local\Temp\ipykernel_11628\578528262.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['8888', 9988, 9999, '-', ' '], np.nan, inplace=True)
C:\Users\IRHAM\AppData\Local\Temp\ipykernel_11628\578528262.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain


Semua data cuaca berhasil digabungkan dan dibersihkan.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATA_TIMESTAMP  4925 non-null   datetime64[ns]
 1   tn              4769 non-null   float64       
 2   tx              4838 non-null   float64       
 3   tavg            4807 non-null   float64       
 4   rh_avg          4809 non-null   float64       
 5   rr              4309 non-null   float64       
 6   ss              4760 non-null   float64       
 7   ff_x            4925 non-null   float64       
 8   ddd_x           4923 non-null   float64       
 9   ff_avg          4628 non-null   float64       
 10  ddd_car         4925 non-null   object        
 11  KABUPATEN       4925 non-null   object        
dtypes: datetime64[ns](1), float64(9), object(2)
memory usage: 461.8+ KB
None


## 4. Geo-tagging dan Agregasi Hotspot (2018-2022)

    Kita memuat data hotspot dari tahun 2018-2022, menandai lokasi kabupatennya berdasarkan data GeoJSON, memfilter hanya untuk 5 wilayah studi, lalu menghitung jumlah hotspot harian.

In [ ]:
hotspot_folder = 'E:/SPARK/SPARK/data/raw/'

# Kita baca data dari 2018 hingga 2023. Data 2023 akan digunakan sebagai "kunci jawaban" nanti.
tahun_untuk_dibaca = range(2018, 2024) # range(2018, 2024) -> 2018, 2019, 2020, 2021, 2022, 2023
hotspot_files = []

for tahun in tahun_untuk_dibaca:
    search_pattern = os.path.join(hotspot_folder, f'hotspot_{tahun}.csv')
    ditemukan = glob.glob(search_pattern)
    if ditemukan:
        hotspot_files.extend(ditemukan)

if not hotspot_files:
    print(f"PERINGATAN: Tidak ada file CSV hotspot ditemukan di '{hotspot_folder}'")
    df_hotspot_harian = pd.DataFrame()
else:
    print(f"Ditemukan {len(hotspot_files)} file hotspot. Menggabungkan...")
    df_hotspot_all = pd.concat((pd.read_csv(f) for f in hotspot_files), ignore_index=True)
    df_hotspot_all['DATA_TIMESTAMP'] = pd.to_datetime(df_hotspot_all['acq_date'])
    
    if not gdf_kalteng.empty:
        gdf_hotspot = gpd.GeoDataFrame(
            df_hotspot_all, geometry=gpd.points_from_xy(df_hotspot_all.longitude, df_hotspot_all.latitude), crs="EPSG:4326"
        )
        df_hotspot_processed = gpd.sjoin(gdf_hotspot, gdf_kalteng, how="inner", predicate='intersects')
        df_hotspot_processed.rename(columns={'NAME_2': 'KABUPATEN'}, inplace=True)
        
        wilayah_studi = [
            'KOTAWARINGIN TIMUR', 'KOTAWARINGIN BARAT', 'PALANGKARAYA', 
            'BARITO UTARA', 'BARITO SELATAN'
        ]
        df_hotspot_processed = df_hotspot_processed[df_hotspot_processed['KABUPATEN'].isin(wilayah_studi)].copy()
        
        df_hotspot_processed.drop(columns=['geometry', 'index_right'], inplace=True)
        print("Geo-tagging, penggantian nama, dan filtering wilayah berhasil.")
        
        df_hotspot_harian = df_hotspot_processed.groupby(
            [pd.Grouper(key='DATA_TIMESTAMP', freq='D'), 'KABUPATEN']
        ).size().reset_index(name='JUMLAH_HOTSPOT')
        print("\nAgregasi data hotspot selesai.")
    else:
        print("Peringatan: Data geospasial tidak tersedia, proses geo-tagging dilewati.")
        df_hotspot_harian = pd.DataFrame()

Ditemukan 6 file hotspot. Menggabungkan...
Geo-tagging, penggantian nama, dan filtering wilayah berhasil.

Agregasi data hotspot selesai.


## 5. Penggabungan Final dan Penyimpanan Dataset

    Menggabungkan data cuaca dan data hotspot yang telah diproses menjadi satu dataset final. Dataset ini kemudian disimpan ke folder processed untuk digunakan nanti.

In [ ]:
output_folder = 'E:/SPARK/SPARK/data/processed/'
output_filename = 'dataset_final_karhutla_kalteng.csv'
full_output_path = os.path.join(output_folder, output_filename)

os.makedirs(output_folder, exist_ok=True)

df_final = pd.merge(
    df_cuaca_all, 
    df_hotspot_harian, 
    on=['DATA_TIMESTAMP', 'KABUPATEN'], 
    how='left'
)

df_final['JUMLAH_HOTSPOT'] = df_final['JUMLAH_HOTSPOT'].fillna(0).astype(int)

df_final.to_csv(full_output_path, index=False)

print(f"\nPROSES SELESAI. Dataset final telah disimpan sebagai '{full_output_path}'")
print(df_final.info())


✅ PROSES SELESAI. Dataset final telah disimpan sebagai 'E:/SPARK/SPARK/data/processed/dataset_final_karhutla_kalteng.csv'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATA_TIMESTAMP  4925 non-null   datetime64[ns]
 1   tn              4769 non-null   float64       
 2   tx              4838 non-null   float64       
 3   tavg            4807 non-null   float64       
 4   rh_avg          4809 non-null   float64       
 5   rr              4309 non-null   float64       
 6   ss              4760 non-null   float64       
 7   ff_x            4925 non-null   float64       
 8   ddd_x           4923 non-null   float64       
 9   ff_avg          4628 non-null   float64       
 10  ddd_car         4925 non-null   object        
 11  KABUPATEN       4925 non-null   object        
 12  JUMLAH_HOTSPOT  4925 non-null   int64

## 6. Feature Engineering dan Persiapan Data untuk Model

    Di sini, kita memuat kembali dataset yang sudah bersih, lalu menyiapkannya untuk model machine learning dengan membuat fitur waktu dan melakukan One-Hot Encoding pada kolom arah angin (ddd_car).

In [23]:
df_final = pd.read_csv(full_output_path)
df_final['DATA_TIMESTAMP'] = pd.to_datetime(df_final['DATA_TIMESTAMP'])

# Membuat fitur baru dari tanggal
df_final['TAHUN'] = df_final['DATA_TIMESTAMP'].dt.year
df_final['BULAN'] = df_final['DATA_TIMESTAMP'].dt.month
df_final['HARI_KE'] = df_final['DATA_TIMESTAMP'].dt.dayofyear

# One-Hot Encoding untuk 'ddd_car'
df_model = pd.get_dummies(df_final, columns=['ddd_car'], prefix='arah_angin', dummy_na=True)

print("\nFeature Engineering selesai. Data siap untuk pemodelan.")
print(df_model.info())


Feature Engineering selesai. Data siap untuk pemodelan.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATA_TIMESTAMP  4925 non-null   datetime64[ns]
 1   tn              4769 non-null   float64       
 2   tx              4838 non-null   float64       
 3   tavg            4807 non-null   float64       
 4   rh_avg          4809 non-null   float64       
 5   rr              4309 non-null   float64       
 6   ss              4760 non-null   float64       
 7   ff_x            4925 non-null   float64       
 8   ddd_x           4923 non-null   float64       
 9   ff_avg          4628 non-null   float64       
 10  KABUPATEN       4925 non-null   object        
 11  JUMLAH_HOTSPOT  4925 non-null   int64         
 12  TAHUN           4925 non-null   int32         
 13  BULAN           4925 non-null   int32         
 14 